# PART 1 - Data Extraction  

## Import neccessary libraries

In [2]:
import numpy as np
import pandas as pd
import lxml
import lxml.etree

## Building Dataframe from webpage

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = pd.read_html(url)
df = data[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Filtering Dataframe to remove 'Not Assigned' Boroughs

In [4]:
filter_df = df.loc[ df['Borough'] != 'Not assigned']
print(f"Not assigned Boroughs: {(filter_df.Borough == 'Not assigned').sum()}")

Not assigned Boroughs: 0


In [5]:
print(f"Not assigned Neighbourhoods: {(filter_df.Neighbourhood == 'Not assigned').sum()}")

Not assigned Neighbourhoods: 0


In [6]:
filter_df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
print(f"Shape of filtered Dataframe: {filter_df.shape}")

Shape of filtered Dataframe: (103, 3)


# PART 2 - Adding Latitudes & Longitudes

## Import neccessary libraries

In [11]:
#!pip install geocoder --quiet
import geocoder
from geopy.geocoders import Nominatim

## Using Zipcodes for Latitudes & Longitudes

In [61]:
zipcodes = filter_df['Postal Code'].to_list()
#lat = []
#long = []

def get_latlong(postal_code, user_agent):
    """
    """

    geolocator = Nominatim(user_agent = user_agent)
    location = geolocator.geocode(f"{postal_code}, Toronto, Ontario, Canada")


    return location.latitude, location.longitude



latitude, longitude = zip(*get_latlong(zipcodes, 'zipcode_to_latlong'))

AttributeError: 'NoneType' object has no attribute 'latitude'

In [67]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
filter_df['location'] = filter_df['Postal Code'].apply(geocode)
filter_df['Latitude'], filter_df['Longitude'] = zip(*filter_df['location'].apply(lambda loc: tuple(loc.latitude, loc.longitude)))
#filter_df['point'] = filter_df['location'].apply(lambda loc: tuple(loc.latitude, loc.longitude))
filter_df.head()

TypeError: tuple expected at most 1 argument, got 2

In [58]:
#TEST Run for single postal code
geolocator = Nominatim(user_agent = 'toronto_zipcodes')
location = geolocator.geocode('M5A')
print([location.latitude, location.longitude])

[45.44058845, 28.018025356938132]
